## Deliverable 2. Create a Customer Travel Destinations Map.

In [39]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps
import numpy as np
# Import API key
from config import gkey

# Configure gmaps API key
gmaps.configure(api_key=gkey)

In [3]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_Database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Alofi,NU,-19.0595,-169.9187,71.49,100,40,2.30,scattered clouds
1,1,Limbang,MY,4.7500,115.0000,80.71,98,100,2.10,overcast clouds
2,2,Jamestown,US,42.0970,-79.2353,72.05,58,39,3.00,scattered clouds
3,3,La Ronge,CA,55.1001,-105.2842,61.02,82,75,5.75,broken clouds
4,4,Geraldton,AU,-28.7667,114.6000,57.61,82,75,5.75,broken clouds


In [4]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 75
What is the maximum temperature you would like for your trip? 90


In [5]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
new_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
new_cities_df.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
1,1,Limbang,MY,4.7500,115.0000,80.71,98,100,2.10,overcast clouds
10,10,Kirakira,SB,-10.4544,161.9205,79.95,84,100,8.41,overcast clouds
13,13,Atuona,PF,-9.8000,-139.0333,77.50,70,68,19.22,light rain
20,20,Albany,US,42.6001,-73.9662,76.30,68,61,1.01,broken clouds
26,26,Namibe,AO,-15.1961,12.1522,77.63,65,0,17.27,clear sky
28,28,Maumere,ID,-8.6199,122.2111,77.88,79,100,7.78,overcast clouds
32,32,San Patricio,US,28.0170,-97.5169,87.69,38,63,17.56,broken clouds
35,35,Warwick,US,41.7001,-71.4162,79.72,76,75,5.75,broken clouds
37,37,Itarema,BR,-2.9248,-39.9167,83.97,67,11,12.26,few clouds
38,38,Butaritari,KI,3.0707,172.7902,81.45,71,66,11.68,broken clouds


In [7]:
# 4a. Determine if there are any empty rows.
new_cities_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 254 entries, 1 to 674
Data columns (total 10 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   City_ID              254 non-null    int64  
 1   City                 254 non-null    object 
 2   Country              253 non-null    object 
 3   Lat                  254 non-null    float64
 4   Lng                  254 non-null    float64
 5   Max Temp             254 non-null    float64
 6   Humidity             254 non-null    int64  
 7   Cloudiness           254 non-null    int64  
 8   Wind Speed           254 non-null    float64
 9   Current Description  254 non-null    object 
dtypes: float64(4), int64(3), object(3)
memory usage: 21.8+ KB


In [11]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
new_cities_df = new_cities_df.dropna()

new_cities_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 253 entries, 1 to 674
Data columns (total 10 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   City_ID              253 non-null    int64  
 1   City                 253 non-null    object 
 2   Country              253 non-null    object 
 3   Lat                  253 non-null    float64
 4   Lng                  253 non-null    float64
 5   Max Temp             253 non-null    float64
 6   Humidity             253 non-null    int64  
 7   Cloudiness           253 non-null    int64  
 8   Wind Speed           253 non-null    float64
 9   Current Description  253 non-null    object 
dtypes: float64(4), int64(3), object(3)
memory usage: 21.7+ KB


In [13]:
new_cities_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
1,1,Limbang,MY,4.7500,115.0000,80.71,98,100,2.10,overcast clouds
10,10,Kirakira,SB,-10.4544,161.9205,79.95,84,100,8.41,overcast clouds
13,13,Atuona,PF,-9.8000,-139.0333,77.50,70,68,19.22,light rain
20,20,Albany,US,42.6001,-73.9662,76.30,68,61,1.01,broken clouds
26,26,Namibe,AO,-15.1961,12.1522,77.63,65,0,17.27,clear sky


In [14]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = new_cities_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
1,Limbang,MY,80.71,overcast clouds,4.7500,115.0000,
10,Kirakira,SB,79.95,overcast clouds,-10.4544,161.9205,
13,Atuona,PF,77.50,light rain,-9.8000,-139.0333,
20,Albany,US,76.30,broken clouds,42.6001,-73.9662,
26,Namibe,AO,77.63,clear sky,-15.1961,12.1522,
28,Maumere,ID,77.88,overcast clouds,-8.6199,122.2111,
32,San Patricio,US,87.69,broken clouds,28.0170,-97.5169,
35,Warwick,US,79.72,broken clouds,41.7001,-71.4162,
37,Itarema,BR,83.97,few clouds,-2.9248,-39.9167,
38,Butaritari,KI,81.45,broken clouds,3.0707,172.7902,


In [16]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": gkey
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    # Add the latitude and longitude to location key for the params dictionary.
    params["location"] = f"{lat},{lng}"
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")
        

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [65]:
# 7. Drop the rows where there is no Hotel Name.
clean_hotel_df = hotel_df.loc[hotel_df["Hotel Name"] != ""]

clean_hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
1,Limbang,MY,80.71,overcast clouds,4.7500,115.0000,Purnama Hotel
10,Kirakira,SB,79.95,overcast clouds,-10.4544,161.9205,SanBiz Lodge
13,Atuona,PF,77.50,light rain,-9.8000,-139.0333,Villa Enata
26,Namibe,AO,77.63,clear sky,-15.1961,12.1522,Hotel Chik Chik Namibe
28,Maumere,ID,77.88,overcast clouds,-8.6199,122.2111,Sylvia Hotel
32,San Patricio,US,87.69,broken clouds,28.0170,-97.5169,"Motel 6 Sinton, TX"
35,Warwick,US,79.72,broken clouds,41.7001,-71.4162,Hampton Inn & Suites Providence/Warwick-Airport
37,Itarema,BR,83.97,few clouds,-2.9248,-39.9167,"Sea Lounge! Bar, Club & Restaurant"
38,Butaritari,KI,81.45,broken clouds,3.0707,172.7902,Isles Sunset Lodge
40,Port Alfred,ZA,77.18,overcast clouds,-33.5906,26.8910,The Halyards Hotel


In [72]:
# 8a. Create the output File (CSV)
output_data_file = "WeatherPy_vacation.csv"
# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [83]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Current Description} and {Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.

locations = clean_hotel_df[["Lat", "Lng"]]

In [87]:
# 11a. Add a marker layer for each city to the map. 
# max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
# heat_layer = gmaps.heatmap_layer(locations, dissipating=False,
             # max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)

fig.add_layer(marker_layer)
# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))